In [ ]:
ç#drive alanına bağlantı
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/hafta8')
!pwd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf # yapay sinir ağları kütüphanesi (sklearn yerine)

In [ ]:
traindata= pd.read_csv('train.csv') # Veri setinin yüklenmesi

In [ ]:
traindata.head()    # csv dosya içeriği

In [ ]:
#Öznitelik Çıkarımı
df_cleaned = traindata.drop(['Survived','PassengerId','Name','Ticket','Cabin'],axis=1)
X = df_cleaned.iloc[:, :].values  # temizlenmiş veriler pclass, sex,age, sibsp, parch, fare, Embarked
y = traindata.iloc[:, 1].values   # Hayatta kalma durumu survived özniteliği

In [ ]:
print(X) # X veri seti bağımsız değişkenler
print(y) # bağımlı değişken

In [ ]:
#Kayıp veri Analizi
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # strateji; boşluk nasıl doldurulacak
imputer.fit(X[:, 2:3])
X[:, 2:3] = imputer.transform(X[:, 2:3])                        # 2. ve 3. sütun sayısal değişkenleri


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')  # kategorikte frekansa bakılır (en çok olan alınır)
imputer.fit(X[:, 6].reshape(-1,1))
X[:, 6] = imputer.transform(X[:, 6].reshape(-1,1)).reshape(-1,)           # 6. sütun kategorik değişkenleri

In [ ]:
#Kategorik verileri sayısala dönüştürdük (Encoding işlemi)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder # ikilik sayı sisteminde vektör atanır

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,4] = le.fit_transform(X[:,4])

In [ ]:
#Train Test verisi ayırma
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
# test size; test için ayrılan veri kümesi
# random state=0 ile veriler sıralı olarak ayrılır. 1 olsaydı rastgele sırada ayırır.

In [ ]:
#Öznitelik ölçeklendirme. Test ve train verilerine standardizasyon işlemi (alogoritması) uygulanır.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) # x test in içine attık, aynı aralığa getirdik; hata vermemesi için

In [ ]:
#Yapay Sinir ağı modeli oluşturma
import tensorflow as tf

ann = tf.keras.models.Sequential()                            # ard arda eklenen veriler için model oluşturma
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))   # 10 tane giriş, aktivasyon fonk.: derleme şekli
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # sigmoid; lojistik regresyona benzer
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# optimizer; ağırlıkları belirleme, loss; hata hesaplama, accuracy; başarı hesaplama

In [ ]:
#Oluşturulan YSA modelinin Fit and predict edilmesi
ann.fit(X_train, y_train, batch_size = 10, epochs = 100) # 100 kez eğitim
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

# her eğitimde optimizer güncelleniyor

In [ ]:
#Test verilerine bakarak eğitim sonucunda oluşturulan ve predict işlemi gerçekleştirilen modelin değerlendirilmesi.
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)